In [1]:
import numpy as np 
import pandas as pd
import torch
import os
from PIL import Image

In [2]:
import torchvision

In [40]:
from torchvision import transforms

In [21]:
from pathlib import Path

In [22]:
from os.path import dirname, abspath

In [28]:
d = dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(abspath("loading_dataset.ipynb")))))))))

In [31]:
os.chdir(d + "/MaskedFace-Net")

In [56]:
transform_train = transforms.Compose(
    [transforms.RandomRotation(20),
     transforms.RandomHorizontalFlip(0.5),
     transforms.CenterCrop(300),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [64]:
transform_val = transforms.Compose(
    [transforms.CenterCrop(300),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [57]:
maskedface_net_train = torchvision.datasets.ImageFolder("../MaskedFace-Net/train", transform=transform_train)

In [67]:
maskedface_net_val = torchvision.datasets.ImageFolder("../MaskedFace-Net/validation", transform=transform_val)

In [74]:
maskedface_net_test = torchvision.datasets.ImageFolder("../MaskedFace-Net/holdout", transform=transform_val)

In [61]:
data_loader_train = torch.utils.data.DataLoader(maskedface_net_train,
                                          batch_size=4,
                                          shuffle=True)

In [68]:
data_loader_val = torch.utils.data.DataLoader(maskedface_net_val,
                                             batch_size=4,
                                             shuffle=True)

In [75]:
data_loader_test = torch.utils.data.DataLoader(maskedface_net_test,
                                             batch_size=4,
                                             shuffle=True)

In [79]:
device = torch.device("cuda")

In [90]:
iter(data_loader_train).next()

[tensor([[[[-0.6784, -0.6941, -0.6863,  ...,  0.5451,  0.5529,  0.5451],
           [-0.6784, -0.7098, -0.6941,  ...,  0.5765,  0.5843,  0.5608],
           [-0.6706, -0.7020, -0.6941,  ...,  0.5686,  0.5843,  0.5922],
           ...,
           [ 0.2392,  0.2471,  0.2549,  ...,  0.0118,  0.0510,  0.2314],
           [ 0.2471,  0.2471,  0.2706,  ...,  0.0980,  0.0353,  0.0745],
           [ 0.2784,  0.2549,  0.2706,  ...,  0.3490,  0.2314,  0.0824]],
 
          [[-0.6941, -0.7098, -0.7020,  ...,  0.1843,  0.1765,  0.1765],
           [-0.6941, -0.7255, -0.7098,  ...,  0.2000,  0.2078,  0.1843],
           [-0.6863, -0.7176, -0.7098,  ...,  0.1922,  0.2078,  0.2000],
           ...,
           [ 0.6941,  0.7020,  0.7098,  ...,  0.4196,  0.4039,  0.3882],
           [ 0.7020,  0.7020,  0.7255,  ...,  0.4353,  0.3647,  0.3725],
           [ 0.7333,  0.7098,  0.7255,  ...,  0.4745,  0.4118,  0.4196]],
 
          [[-0.7961, -0.8118, -0.7961,  ...,  0.0667,  0.0667,  0.0510],
           [-

In [10]:
def get_resnet():
    input_img = Input((1,28,28),name='input_layer')
    zeroPad1 = ZeroPadding2D((1,1), name='zeroPad1', dim_ordering='th')
    zeroPad1_2 = ZeroPadding2D((1,1), name='zeroPad1_2', dim_ordering='th')
    layer1 = Convolution2D(6, 3, 3, subsample=(2, 2), init='he_uniform', name='major_conv', dim_ordering='th')
    layer1_2 = Convolution2D(16, 3, 3, subsample=(2, 2), init='he_uniform', name='major_conv2', dim_ordering='th')
    zeroPad2 = ZeroPadding2D((1,1), name='zeroPad2', dim_ordering='th')
    zeroPad2_2 = ZeroPadding2D((1,1), name='zeroPad2_2', dim_ordering='th')
    layer2 = Convolution2D(6, 3, 3, subsample=(1,1), init='he_uniform', name='l1_conv', dim_ordering='th')
    layer2_2 = Convolution2D(16, 3, 3, subsample=(1,1), init='he_uniform', name='l1_conv2', dim_ordering='th')
    zeroPad3 = ZeroPadding2D((1,1), name='zeroPad3', dim_ordering='th')
    zeroPad3_2 = ZeroPadding2D((1,1), name='zeroPad3_2', dim_ordering='th')
    layer3 = Convolution2D(6, 3, 3, subsample=(1, 1), init='he_uniform', name='l2_conv', dim_ordering='th')
    layer3_2 = Convolution2D(16, 3, 3, subsample=(1, 1), init='he_uniform', name='l2_conv2', dim_ordering='th')
    layer4 = Dense(64, activation='relu', init='he_uniform', name='dense1')
    layer5 = Dense(16, activation='relu', init='he_uniform', name='dense2')
    final = Dense(10, activation='softmax', init='he_uniform', name='classifier')
    
    sgd = SGD(decay=0., lr=0.01, momentum=0.9, nesterov=True)
    model.compile(loss=scc, optimizer=sgd, metrics=['accuracy'])
    return model